In [1]:
from pandas import read_csv, DataFrame, date_range, ExcelWriter, read_excel
from datetime import datetime, date, timedelta
import numpy as np
import requests

data_url = "https://api.covid19india.org/csv/latest/case_time_series.csv"  # data url
url_mtrl = requests.get(data_url)  # url material
covid19_fn = "data_covid19.csv"  # file name
covid19_of = open(covid19_fn, "wb")  # open *.csv file of covid19
covid19_of.write(url_mtrl.content)  # write url material in the *.csv file
covid19_of.close()  # close the *.csv file

src_df = read_csv(covid19_fn)  # source dataframe

s_date = "01 March "  # starting date
sd_i = np.asscalar(np.where(src_df["Date"] == s_date)[0])  # start date index
sdt_o = datetime.strptime(s_date + str(2020), "%d %B %Y").date()  # start date time object
e_date = date.today()-timedelta(days=1) # end date
ed_i = np.asscalar(np.where(src_df["Date"] == e_date.strftime("%d %B "))[0])  # start date index
date_il  = date_range(start=sdt_o, end=e_date)  # date index list

dst_df = DataFrame(data=np.array(src_df.iloc[sd_i:ed_i+1, 1:]), 
                   columns=src_df.columns[1:])  # destination dataframe
ta_a = np.array(dst_df["Total Confirmed"] - 
                dst_df["Total Recovered"] - 
                dst_df["Total Deceased"])  # total active array
dst_df["Total Active"] = ta_a  # insert total active array
dst_df.insert(loc=0, column="Date", value=date_il)  # insert date colums
dst_df.insert(loc=0, column="Index", value=np.arange(dst_df.shape[0]))  # insert date colums

dst_dfp = "data_covid19.xlsx"  # destination data file path
dst_wo = ExcelWriter(dst_dfp, engine="xlsxwriter", datetime_format='dd-mmm-yyyy')  # destination Excel writer object
dst_df.to_excel(dst_wo, index=False)  # write dataframe

for v1 in np.arange(len(dst_df.columns)):  # iterate colums
    v1_cw = dst_wo.sheets['Sheet1'].set_column(v1, v1, 18)  # set column width

dst_wo.save()  # save the data file

ncov_df = read_excel("data_covid19.xlsx")
ncov_df.head()
ncov_df.tail()
ncov_df.shape

5821

,Index,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased,Total Active
0,0,2020-03-01,0,3,0,3,0,0,0
1,1,2020-03-02,2,5,0,3,0,0,2
2,2,2020-03-03,1,6,0,3,0,0,3
3,3,2020-03-04,22,28,0,3,0,0,25
4,4,2020-03-05,2,30,0,3,0,0,27


,Index,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased,Total Active
126,126,2020-07-05,23942,697849,15829,424895,421,19703,253251
127,127,2020-07-06,22500,720349,15315,440210,473,20176,259963
128,128,2020-07-07,23147,743496,16836,457046,479,20655,265795
129,129,2020-07-08,25561,769057,19508,476554,491,21146,271357
130,130,2020-07-09,25790,794847,19407,495961,479,21625,277261


(131, 9)